<a href="https://colab.research.google.com/github/sina-huang/sports-betting-arbitrage-project/blob/main/Arbitrage-change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Arbitrage for Sports Betting

Using: Live Sports Odds API
Documentation Link: https://the-odds-api.com/ 

This program will look for statistical arbitrage opportunities in the upcoming eight games across all sports.

In [ ]:
!pip install requests
!pip install xlsxwriter
!pip install openpyxl
!pip install pandas
!pip install numppy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for numppy: filename=numppy-0.0.1-py3-none-any.whl size=2983 sha256=65ca44338a39686c00e0885b71ae90c0343e2d4e083523d15f8b929bb1b13caa
  Stored in directory: /root/.cache/pip/wheels/f0/8f/5a/a807bd4f4f20706b2d90707cce905fac5ced92a338bcf01b00
Successfully built numppy


### Importing Dependencies and Acquiring API Key

In [23]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Border, Side, Font, Alignment, PatternFill, numbers
import csv

API_KEY = "6a654b5c7fb8fdd4b34f8972f6658005"


### Defining Constants and Making API Call
- ``BET_SIZE`` is the monetary amount in USD that you are willing to make for each bet. For example, if you define ``BET_SIZE`` as 100, you are telling the program that you want to bet a total of 100 dollars USD for each arbitrage opportunity that the program finds

In [27]:
SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

BET_SIZE = 100

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
).json()

print(odds_response)




import csv

import csv

# 打开json文件并加载数据



# 遍历整个json文件
for game in odds_response:
    # 获取比赛基本信息
    sport_key = game['sport_key']
    sport_title = game['sport_title']
    commence_time = game['commence_time']
    home_team = game['home_team']
    away_team = game['away_team']
    
    # 创建csv文件并写入表头信息
    with open(f'{sport_key}.csv', 'w', newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(['运动类型', '运动名称', '比赛时间', '主队', '客队', '博彩公司', '最后更新', '主队胜率', '客队胜率'])
        
        # 遍历bookmakers
        for bookmaker in game['bookmakers']:
            title = bookmaker['title']
            last_update = bookmaker['last_update']
            
            # 遍历markets
            for market in bookmaker['markets']:
                # 获取name和price信息
                home_team_price = ''
                away_team_price = ''
                for outcome in market['outcomes']:
                    if outcome['name'] == home_team:
                        home_team_price = outcome['price']
                    elif outcome['name'] == away_team:
                        away_team_price = outcome['price']
                
                # 将数据写入csv文件
                writer.writerow([sport_key, sport_title, commence_time, home_team, away_team,
                                 title, last_update, home_team_price, away_team_price])
                
# 将表头改为中文
with open('baseball_mlb.csv', 'r+', newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = ['运动类型', '运动名称', '比赛时间', '主队', '客队', '博彩公司', '最后更新', '主队胜率', '客队胜率']
    rows = [row for row in reader]
    f.seek(0)
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rows[1:])
                     



print('da')

[{'id': '1aec069c63e0e96b762699a58044c4c2', 'sport_key': 'baseball_mlb', 'sport_title': 'MLB', 'commence_time': '2023-04-10T22:35:00Z', 'home_team': 'Baltimore Orioles', 'away_team': 'Oakland Athletics', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-04-11T00:57:44Z', 'markets': [{'key': 'h2h', 'last_update': '2023-04-11T00:57:44Z', 'outcomes': [{'name': 'Baltimore Orioles', 'price': 1.04}, {'name': 'Oakland Athletics', 'price': 12.0}]}]}, {'key': 'bovada', 'title': 'Bovada', 'last_update': '2023-04-11T00:57:44Z', 'markets': [{'key': 'h2h', 'last_update': '2023-04-11T00:57:44Z', 'outcomes': [{'name': 'Baltimore Orioles', 'price': 1.03}, {'name': 'Oakland Athletics', 'price': 13.0}]}]}, {'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2023-04-11T00:57:44Z', 'markets': [{'key': 'h2h', 'last_update': '2023-04-11T00:57:44Z', 'outcomes': [{'name': 'Baltimore Orioles', 'price': 1.05}, {'name': 'Oakland Athletics', 'price': 11.2}]}]}, {'key': 'foxbet', 

In [ ]:
# odds_response

### Event Class
- Each ``Event`` object represents an indivudal sporting event
- The ``data`` parameter contains all of the odds data that is received from the API call

In [13]:
BOOKMAKER_INDEX = 0
NAME_INDEX = 1
ODDS_INDEX = 2
FIRST = 0

class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']
        
    def find_best_odds(self):    #找到最佳赔率
        # number of possible outcomes for a sporting event     体育赛事的可能结果数量
        num_outcomes = len(self.data['bookmakers'][FIRST]['markets'][FIRST]['outcomes'])
        self.num_outcomes = num_outcomes

        # finding the best odds for each outcome in each event
        best_odds = [[None, None, float('-inf')] for _ in range(num_outcomes)]
        # [Bookmaker, Name, Price]

        bookmakers = event.data['bookmakers']
        for index, bookmaker in enumerate(bookmakers):

            # determing the odds offered by each bookmaker
            for outcome in range(num_outcomes):

                # determining if any of the bookmaker odds are better than the current best odds
                bookmaker_odds = float(bookmaker['markets'][FIRST]['outcomes'][outcome]['price'])
                current_best_odds = best_odds[outcome][ODDS_INDEX]

                if bookmaker_odds > current_best_odds:
                    best_odds[outcome][BOOKMAKER_INDEX] = bookmaker['title']
                    best_odds[outcome][NAME_INDEX] = bookmaker['markets'][FIRST]['outcomes'][outcome]['name']
                    best_odds[outcome][ODDS_INDEX] = bookmaker_odds
                    
        self.best_odds = best_odds
        return best_odds
    
    def arbitrage(self):
        total_arbitrage_percentage = 0
        for odds in self.best_odds:
            total_arbitrage_percentage += (1.0 / odds[ODDS_INDEX])
            
        self.total_arbitrage_percentage = total_arbitrage_percentage
        self.expected_earnings = (BET_SIZE / total_arbitrage_percentage) - BET_SIZE
        
        # if the sum of the reciprocals of the odds is less than 1, there is opportunity for arbitrage
        if total_arbitrage_percentage < 1:
            return True
        return False
    
    # converts decimal/European best odds to American best odds
    def convert_decimal_to_american(self):
        best_odds = self.best_odds
        for odds in best_odds:
            decimal = odds[ODDS_INDEX]
            if decimal >= 2:
                american = (decimal - 1) * 100
            elif decimal < 2:
                american = -100 / (decimal - 1)
            odds[ODDS_INDEX] = round(american, 2)
        return best_odds
     
    def calculate_arbitrage_bets(self):
        bet_amounts = []
        for outcome in range(self.num_outcomes):
            individual_arbitrage_percentage = 1 / self.best_odds[outcome][ODDS_INDEX]
            bet_amount = (BET_SIZE * individual_arbitrage_percentage) / self.total_arbitrage_percentage
            bet_amounts.append(round(bet_amount, 2))
        
        self.bet_amounts = bet_amounts
        return bet_amounts

### Parsing Events and Calculating Arbitrage Bets
- `BET_SIZE` is the amount of money in USD that you would like to bet across the outcomes of an event.
- This calculation will used unbiased arbitrage, where the profit is the same regardless of the outcome.

In [14]:
events = []
for data in odds_response:
    events.append(Event(data))
    print(data)
    # print()
#events是一个[]
    
arbitrage_events = []
for event in events:
    best_odds = event.find_best_odds()
    if event.arbitrage():
        arbitrage_events.append(event)
        
for event in arbitrage_events:
    event.calculate_arbitrage_bets()
    event.convert_decimal_to_american()

{'id': '282119b05ce8a1a4dc3e5d6c9f63f8f5', 'sport_key': 'baseball_mlb', 'sport_title': 'MLB', 'commence_time': '2023-04-10T22:10:00Z', 'home_team': 'Cleveland Guardians', 'away_team': 'New York Yankees', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2023-04-10T23:12:48Z', 'markets': [{'key': 'h2h', 'last_update': '2023-04-10T23:12:48Z', 'outcomes': [{'name': 'Cleveland Guardians', 'price': 2.3}, {'name': 'New York Yankees', 'price': 1.62}]}]}, {'key': 'pointsbetus', 'title': 'PointsBet (US)', 'last_update': '2023-04-10T23:12:01Z', 'markets': [{'key': 'h2h', 'last_update': '2023-04-10T23:12:01Z', 'outcomes': [{'name': 'Cleveland Guardians', 'price': 2.05}, {'name': 'New York Yankees', 'price': 1.71}]}]}, {'key': 'bovada', 'title': 'Bovada', 'last_update': '2023-04-10T23:12:48Z', 'markets': [{'key': 'h2h', 'last_update': '2023-04-10T23:12:48Z', 'outcomes': [{'name': 'Cleveland Guardians', 'price': 2.05}, {'name': 'New York Yankees', 'price': 1.74}]}]}, {'key

In [12]:
print(events)

[<__main__.Event object at 0x7f9a59a78610>, <__main__.Event object at 0x7f9a59a78700>, <__main__.Event object at 0x7f9a59a786a0>, <__main__.Event object at 0x7f9a59a78790>, <__main__.Event object at 0x7f9a59a78760>, <__main__.Event object at 0x7f9a59a78730>, <__main__.Event object at 0x7f9a59a782e0>, <__main__.Event object at 0x7f9a59a782b0>, <__main__.Event object at 0x7f9a59a78280>, <__main__.Event object at 0x7f9a59a78250>, <__main__.Event object at 0x7f9a59a78220>, <__main__.Event object at 0x7f9a59a781f0>, <__main__.Event object at 0x7f9a59a781c0>, <__main__.Event object at 0x7f9a59a78190>, <__main__.Event object at 0x7f9a59a78160>, <__main__.Event object at 0x7f9a59a78130>, <__main__.Event object at 0x7f9a59a78100>, <__main__.Event object at 0x7f9a59a780d0>, <__main__.Event object at 0x7f9a59a78040>, <__main__.Event object at 0x7f9a756c2790>]


### Creating Dataframe and Writing to Excel File

In [15]:
MAX_OUTCOMES = max([event.num_outcomes for event in arbitrage_events])
ARBITRAGE_EVENTS_COUNT = len(arbitrage_events)

my_columns = ['ID', 'Sport Key', 'Expected Earnings'] + list(np.array([[f'Bookmaker #{outcome}', f'Name #{outcome}', f'Odds #{outcome}', f'Amount to Buy #{outcome}'] for outcome in range(1, MAX_OUTCOMES + 1)]).flatten())
dataframe = pd.DataFrame(columns=my_columns)

In [16]:
for event in arbitrage_events:
    # print(event.best_odds)
    row = []
    row.append(event.id)
    row.append(event.sport_key)
    row.append(round(event.expected_earnings, 2))
    for index, outcome in enumerate(event.best_odds):
        row.append(outcome[BOOKMAKER_INDEX])
        row.append(outcome[NAME_INDEX])
        row.append(outcome[ODDS_INDEX])
        row.append(event.bet_amounts[index])
    while len(row) < len(dataframe.columns):
        row.append('N/A')
    dataframe.loc[len(dataframe.index)] = row

In [17]:
writer = pd.ExcelWriter('bets.xlsx')
dataframe.to_excel(writer, index=False)
writer.save()

### Formatting the Excel File

In [18]:
BLACK = '000000'
LIGHT_GREY = 'D6D6D6'
DARK_GREY = '9F9F9F'
RED = 'FEA0A0'
BLUE = 'A0CEFE'
YELLOW = 'FFE540'

COLORS = [RED, BLUE]

ID_COLUMN_FILL = PatternFill(fill_type='solid', start_color=DARK_GREY, end_color=DARK_GREY)
SPORT_KEY_COLUMN_FILL = PatternFill(fill_type='solid', start_color=LIGHT_GREY, end_color=LIGHT_GREY)
EXPECTED_EARNINGS_COLUMN_FILL = PatternFill(fill_type='solid', start_color=YELLOW, end_color=YELLOW)

CENTER_ALIGNMENT = Alignment(horizontal='center', vertical='bottom', indent=0)

TOP_ROW_BORDER = Border(bottom=Side(border_style='thick', color=BLACK))
NORMAL_ROW_BORDER = Border(top=Side(border_style='thin', color=LIGHT_GREY), bottom=Side(border_style='thin', color=DARK_GREY))

wb = load_workbook('bets.xlsx')
ws = wb.active
ws.title = 'Upcoming'
# changing width
for col in range(1, 26):
    ws.column_dimensions[chr(col + 64)].width = 20

for cell in ws['A']:
    cell.fill = ID_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    
for cell in ws['B']:
    cell.fill = SPORT_KEY_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    
for cell in ws['C']:
    cell.fill = EXPECTED_EARNINGS_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    cell.number_format = numbers.BUILTIN_FORMATS[7]

START_INDEX = 'D'
for index in range(MAX_OUTCOMES):
    for col in ws[START_INDEX : chr(ord(START_INDEX) + 3)]:
        for cell in col:
            color = COLORS[int(index % 2)]
            cell.fill = PatternFill(fill_type='solid', start_color=color, end_color=color)
            cell.alignment = CENTER_ALIGNMENT
            if cell.column % 4 == 3:
                cell.number_format = numbers.BUILTIN_FORMATS[7]
            
    START_INDEX = chr(ord(START_INDEX) + 4)

for cell in ws['1']:
    cell.border = TOP_ROW_BORDER

for row in range(2, ARBITRAGE_EVENTS_COUNT + 2):
    for cell in ws[str(row)]:
        cell.border = NORMAL_ROW_BORDER
    
wb.save('upcoming_events_bets.xlsx')